# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [9]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

In [10]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os

    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [11]:
from pyspark.sql import SparkSession

# Initialize a Spark session with local mode
spark = SparkSession.builder.appName("LocalClusterApp").master("local[*]").getOrCreate()


# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [12]:
# Create a sample DataFrame
df = spark.read.csv("netflix-rotten-tomatoes-metacritic-imdb.csv", header=True)
df.show(5)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

## What is the maximum and average of the overall hidden gem score?

In [44]:
df.select("Hidden Gem Score").summary("max", "mean").show()

+-------+-----------------+
|summary| Hidden Gem Score|
+-------+-----------------+
|    max|              9.8|
|   mean|5.937551386501234|
+-------+-----------------+



## How many movies that are available in Korea?

In [66]:
from pyspark.sql.functions import col

df.select("Country Availability").filter(col("Country Availability").contains("Korea")).count()

4845

## Which director has the highest average hidden gem score?

In [15]:
from pyspark.sql.functions import avg

df.groupBy("Director").agg(avg("Hidden Gem Score")).sort("avg(Hidden Gem Score)", ascending=False).show(1)

+-----------+---------------------+
|   Director|avg(Hidden Gem Score)|
+-----------+---------------------+
|Dorin Marcu|                  9.8|
+-----------+---------------------+
only showing top 1 row



## How many genres are there in the dataset?

In [37]:
from pyspark.sql.functions import split, explode

df_genres = df.withColumn("Genre", split(df["Genre"], ", "))
df_genres_exploded = df_genres.select(explode("Genre").alias("Genre"))
df_genres_exploded.select("Genre").distinct().count()

28